In [35]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [36]:
import numpy as np
import pandas as pd
from scipy import sparse
import xgboost as xgb
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

train = pd.read_csv("data/train_all_feat.csv")
test  = pd.read_csv("data/test_all_feat.csv")

target_num_map = {'high':0, 'medium':1, 'low':2}
train_y = pd.read_csv("data/y.csv")
train_y = np.array(train_y['interest_level'].apply(lambda x: target_num_map[x]))

features_to_use=["bathrooms", "bedrooms", "latitude", "longitude", "price", "num_features", 
    "len_description","created_year", "created_month", "created_day", "created_hour", "created_weekday", 
    "is_night", "is_weekend", "price_per_bed", "price_per_bath", "total_rooms", "price_per_room",
    "manager_skill", "building_popularity", "logprice", "density"]

count_features = CountVectorizer(stop_words='english', max_features=500,ngram_range=(1,4))
count_features_train_sparse = count_features.fit_transform(train["features"])
count_features_test_sparse = count_features.transform(test["features"])

train_X = sparse.hstack([train[features_to_use], count_features_train_sparse]).tocsr()
test_X = sparse.hstack([test[features_to_use], count_features_test_sparse]).tocsr()

tfidf_features = TfidfVectorizer(stop_words='english', max_features=500,ngram_range=(1,4))
tfidf_features_train_sparse = count_features.fit_transform(train_df["features"])
tfidf_features_test_sparse = count_features.transform(test_df["features"])

train_X = sparse.hstack([train_X, tfidf_features_train_sparse]).tocsr()
test_X = sparse.hstack([test_X, tfidf_features_test_sparse]).tocsr()

count_desc = CountVectorizer(stop_words='english', max_features=500,ngram_range=(1,4))
count_desc_train_sparse = count_desc.fit_transform(train_df["description"])
count_desc_test_sparse = count_desc.transform(test_df["description"])

train_X = sparse.hstack([train_X, count_desc_train_sparse]).tocsr()
test_X = sparse.hstack([test_X, count_desc_test_sparse]).tocsr()

tfidf_desc = CountVectorizer(stop_words='english', max_features=500,ngram_range=(1,4))
tfidf_desc_train_sparse = tfidf_desc.fit_transform(train_df["description"])
tfidf_desc_test_sparse = tfidf_desc.transform(test_df["description"])

train_X = sparse.hstack([train_X, tfidf_desc_train_sparse]).tocsr()
test_X = sparse.hstack([test_X, tfidf_desc_test_sparse]).tocsr()

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [22]:
X = pd.read_csv('data/X.csv')
y = np.asarray(pd.read_csv('data/y.csv')).ravel()

X = X.drop(['Unnamed: 0'], axis=1)

print(X.head())
print(y)

   bathrooms  bedrooms  latitude  longitude   price  num_features  \
0        1.5         3   40.7145   -73.9425  3000.0           0.0   
1        1.0         1   40.7388   -74.0018  2850.0           4.0   
2        1.0         1   40.7539   -73.9677  3275.0           2.0   
3        2.0         4   40.7429   -74.0028  7995.0           0.0   
4        1.0         1   40.8234   -73.9457  1725.0           4.0   

   len_description  created_year  created_month  created_day  \
0            561.0          2016              6           24   
1            657.0          2016              4           17   
2            404.0          2016              4           18   
3              8.0          2016              4           19   
4             22.0          2016              4           20   

             ...             price_per_bed  price_per_bath  total_rooms  \
0            ...                     750.0          1200.0          4.5   
1            ...                    1425.0        

{'min_impurity_split': None, 'n_jobs': 1, 'n_estimators': 10, 'min_samples_leaf': 1, 'bootstrap': True, 'max_depth': None, 'warm_start': False, 'oob_score': False, 'verbose': 0, 'random_state': None, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'class_weight': None, 'min_impurity_decrease': 0.0, 'max_leaf_nodes': None, 'criterion': 'gini', 'max_features': 'auto'}


In [30]:
model = RandomForestClassifier()
print(model.get_params())

grid = {'n_estimators' : [200, 350, 500, 750, 1000],
        'max_depth': [int(x) for x in np.linspace(10, 110, 11)],
        'min_samples_split': [2, 5, 10, 25, 50],
        'min_samples_leaf': [2, 5, 10, 25, 50],
        }
#'max_feaures': ['sqrt', 'log2']
#'bootstrap': [True, False]

kFold = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)
randomSearch = RandomizedSearchCV(estimator=model, param_distributions=grid, n_iter=50, 
                                  verbose=2, scoring="neg_log_loss", n_jobs=-1, cv=kFold)

rs_result = randomSearch.fit(X, y)

print("Best: %f using %s" % (rs_result.best_score_, rs_result.best_params_))
means = rs_result.cv_results_['mean_test_score']
stds = rs_result.cv_results_['std_test_score']
params = rs_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] min_samples_split=10, max_depth=10, n_estimators=1000, min_samples_leaf=5 
[CV] min_samples_split=10, max_depth=10, n_estimators=1000, min_samples_leaf=5 
[CV] min_samples_split=10, max_depth=10, n_estimators=1000, min_samples_leaf=5 
[CV] min_samples_split=2, max_depth=110, n_estimators=750, min_samples_leaf=2 
[CV]  min_samples_split=10, max_depth=10, n_estimators=1000, min_samples_leaf=5, total= 1.0min
[CV] min_samples_split=2, max_depth=110, n_estimators=750, min_samples_leaf=2 
[CV]  min_samples_split=10, max_depth=10, n_estimators=1000, min_samples_leaf=5, total= 1.0min
[CV] min_samples_split=2, max_depth=110, n_estimators=750, min_samples_leaf=2 
[CV]  min_samples_split=10, max_depth=10, n_estimators=1000, min_samples_leaf=5, total= 1.0min
[CV] min_samples_split=50, max_depth=10, n_estimators=500, min_samples_leaf=2 
[CV]  min_samples_split=2, max_depth=110, n_estimators=750, min_samples_leaf=2, total= 1.0min
[CV

KeyboardInterrupt: 